<div align="center">

# 🔍 Uninformed/Informed Search Techniques

###  **Artin Tavasoli** 👋🏻 
📘 **Student ID:** `810102543`

</div>


In [83]:
from game import Game
import time
import os
import re

## Load maps

In [84]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [85]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [86]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [87]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [88]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [89]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [90]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [91]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

In [92]:
class State:
    def __init__(self, pos_player, pos_boxes, direction_from_father):
        self.pos_player = pos_player
        self.pos_boxes = pos_boxes
        self.direction_from_father = direction_from_father
        self.heuristic = 0
        self.depth = 0
    def get_player_pos(self):
        return self.pos_player
    def get_boxes_pos(self):
        return self.pos_boxes
    def get_solution(self):
        if self.father is None:
            return ""
        return self.father.get_solution() + self.direction_from_father
    def __lt__(self, other):
        return  self.sum_hurisitic_and_cost_spent < other.sum_hurisitic_and_cost_spent
    def set_depth(self, depth):
        self.depth = depth
    def get_depth(self):
        return self.depth
    def set_sum_hurisitic_and_cost_spent(self, sum_hurisitic_and_cost_spent):
        self.sum_hurisitic_and_cost_spent = sum_hurisitic_and_cost_spent
    def set_father(self, father):
        self.father = father
        
    
def get_state_id(player_pos, boxes_pos):
    return hash(str(player_pos) + str(boxes_pos))

### BFS

In [ ]:
from collections import deque
def solver_bfs(game_map):
    game = Game(game_map)
    frontier = deque()
    explored = set()
    root = State(game.get_player_position(),game.get_box_locations(),"")
    root.set_father(None)
    frontier.append(root)
    while frontier:
        the_node = frontier.popleft()
        explored.add(get_state_id(the_node.get_player_pos(), the_node.get_boxes_pos()))
        for direction in ['U', 'D', 'R', 'L']:
            game.set_player_position(the_node.get_player_pos())
            game.set_box_positions(the_node.get_boxes_pos())
            result = game.apply_move(direction)
            if(result):
                child = State(game.get_player_position(),game.get_box_locations(),direction)
                child.set_father(the_node)
                child_state_id = get_state_id(child.get_player_pos(), child.get_boxes_pos())
                if game.is_game_won():
                    return child.get_solution(), len(explored)
                if child_state_id not in explored:
                    frontier.append(child)
                    explored.add(child_state_id)
    return None,len(explored)

### DFS

In [94]:
def dfs(the_node, visited, game, number_states_seen):
    number_states_seen += 1
    game.set_player_position(the_node.get_player_pos())
    game.set_box_positions(the_node.get_boxes_pos())
    the_node_state_id = get_state_id(the_node.get_player_pos(), the_node.get_boxes_pos())
    visited.add(the_node_state_id)
    if game.is_game_won():
        return the_node.get_solution(), number_states_seen
    for direction in ['U', 'D', 'R', 'L']:
        game.set_player_position(the_node.get_player_pos())
        game.set_box_positions(the_node.get_boxes_pos())
        result = game.apply_move(direction)
        if result:
            child = State(game.get_player_position(), game.get_box_locations(), direction)
            child.set_father(the_node)
            child_state_id = get_state_id(child.get_player_pos(), child.get_boxes_pos()) 
            if child_state_id not in visited:
                ans, number_states_seen = dfs(child, visited, game, number_states_seen)
                if ans:
                    return ans, number_states_seen
    visited.remove(the_node_state_id)
    return None, number_states_seen

def solver_dfs(game_map):
    game = Game(game_map)
    visited = set()
    root = State(game.get_player_position(), game.get_box_locations(), "")
    root.set_father(None)
    return dfs(root, visited, game, 0)

### IDS

In [95]:
def ids(the_node, visited, game, number_states_seen, depth_restricted):
    if the_node.get_depth() > depth_restricted:
        return None, number_states_seen
    number_states_seen += 1
    game.set_player_position(the_node.get_player_pos())
    game.set_box_positions(the_node.get_boxes_pos())
    the_node_state_id = get_state_id(the_node.get_player_pos(), the_node.get_boxes_pos())
    visited.add(the_node_state_id)
    if game.is_game_won():
        return the_node.get_solution(), number_states_seen
    for direction in ['U', 'D', 'R', 'L']:
        game.set_player_position(the_node.get_player_pos())
        game.set_box_positions(the_node.get_boxes_pos())
        result = game.apply_move(direction)
        if result:
            child = State(game.get_player_position(), game.get_box_locations(), direction)
            child.set_father(the_node)
            child.set_depth(the_node.get_depth() + 1)
            child_state_id = get_state_id(child.get_player_pos(), child.get_boxes_pos()) 
            if child_state_id not in visited:
                ans, number_states_seen = ids(child, visited, game, number_states_seen, depth_restricted)
                if ans:
                    return ans, number_states_seen
    visited.remove(the_node_state_id)
    return None, number_states_seen

def solver_ids(game_map):
    game = Game(game_map)
    visited = set()
    root = State(game.get_player_position(),game.get_box_locations(),"")
    root.set_depth(0)
    root.set_father(None)
    for depth_restricted in range(1000):
        ans,number_states_seen = ids(root, visited, game, 0, depth_restricted)
        if ans:
            return ans,number_states_seen
    return None, number_states_seen

### A*

HEURISTIC

In [96]:
from heapq import heappush, heappop
def manhattan_dist(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])    

def heuristic0(game):
    return 0

def heuristic1(game):
    box_positions = game.get_box_locations()
    goal_positions = game.get_goal_locations()
    heuristic = 0
    for i in range(len(box_positions)):
        dist_box_from_goal = manhattan_dist(box_positions[i], goal_positions[i])
        heuristic += dist_box_from_goal
    return heuristic

def calculate_min_dist_box_to_portal_to_goal(portal_locations, box_pos, goal_pos):
    ans = 1e9
    for j in range(len(portal_locations)):
        dist1 = manhattan_dist(box_pos, portal_locations[j][0]) + manhattan_dist(portal_locations[j][1], goal_pos)
        dist2 = manhattan_dist(box_pos, portal_locations[j][1]) + manhattan_dist(portal_locations[j][0], goal_pos)
        dist_box_from_goal_with_portals = min(dist1, dist2)
        if dist_box_from_goal_with_portals < ans:
            ans = dist_box_from_goal_with_portals
    return ans

def heuristic2(game):
    box_positions = game.get_box_locations()
    goal_positions = game.get_goal_locations()
    portal_locations = game.get_portal_locations()
    heuristic = 0
    for i in range(len(box_positions)):
        dist_box_from_goal = manhattan_dist(box_positions[i], goal_positions[i])
        dist_box_from_goal_with_portal = calculate_min_dist_box_to_portal_to_goal(portal_locations, box_positions[i], goal_positions[i])
        heuristic += min(dist_box_from_goal, dist_box_from_goal_with_portal)  
    return heuristic

A* with 0 as heuristic

In [97]:
def solver_astar(game_map, heuristic_func=heuristic0, weight=1):
    game = Game(game_map)
    root = State(game.get_player_position(),game.get_box_locations(),"")
    frontier = []
    explored = set()
    best_cost = {}
    root.set_depth(0)
    root_cost = weight * heuristic_func(game)
    root.set_sum_hurisitic_and_cost_spent(root_cost)
    root.set_father(None)
    heappush(frontier, root)
    best_cost[get_state_id(root.get_player_pos(), root.get_boxes_pos())] = root_cost
    while frontier:
        the_node = heappop(frontier)
        explored.add(get_state_id(the_node.get_player_pos(), the_node.get_boxes_pos()))
        game.set_player_position(the_node.get_player_pos())
        game.set_box_positions(the_node.get_boxes_pos())
        if game.is_game_won():
            return the_node.get_solution(), len(explored)
        for direction in ['U', 'D', 'R', 'L']:
            game.set_player_position(the_node.get_player_pos())
            game.set_box_positions(the_node.get_boxes_pos())
            result = game.apply_move(direction)
            if(result):
                child_state_id = get_state_id(game.get_player_position(), game.get_box_locations())
                if child_state_id not in explored:
                    child_cost = weight * heuristic_func(game) + the_node.get_depth() + 1
                    if child_state_id not in best_cost or child_cost < best_cost[child_state_id]:
                        best_cost[child_state_id] = child_cost
                        child = State(game.get_player_position(), game.get_box_locations(), direction)
                        child.set_sum_hurisitic_and_cost_spent(child_cost)
                        child.set_depth(the_node.get_depth() + 1)
                        child.set_father(the_node)
                        heappush(frontier, child)           
    return None, len(explored)

A* with box-goal heuristic

this is not a good heuristic for when you have portals, the next heuristic is the best one

In [98]:
def solver_astar2(game_map, heuristic_func=heuristic1, weight=1):
    game = Game(game_map)
    root = State(game.get_player_position(),game.get_box_locations(),"")
    frontier = []
    explored = set()
    best_cost = {}
    root.set_depth(0)
    root_cost = weight * heuristic_func(game)
    root.set_sum_hurisitic_and_cost_spent(root_cost)
    root.set_father(None)
    heappush(frontier, root)
    best_cost[get_state_id(root.get_player_pos(), root.get_boxes_pos())] = root_cost
    while frontier:
        the_node = heappop(frontier)
        explored.add(get_state_id(the_node.get_player_pos(), the_node.get_boxes_pos()))
        game.set_player_position(the_node.get_player_pos())
        game.set_box_positions(the_node.get_boxes_pos())
        if game.is_game_won():
            return the_node.get_solution(), len(explored)
        for direction in ['U', 'D', 'R', 'L']:
            game.set_player_position(the_node.get_player_pos())
            game.set_box_positions(the_node.get_boxes_pos())
            result = game.apply_move(direction)
            if(result):
                child_state_id = get_state_id(game.get_player_position(), game.get_box_locations())
                if child_state_id not in explored:
                    child_cost = weight * heuristic_func(game) + the_node.get_depth() + 1
                    if child_state_id not in best_cost or child_cost < best_cost[child_state_id]:
                        best_cost[child_state_id] = child_cost
                        child = State(game.get_player_position(), game.get_box_locations(), direction)
                        child.set_sum_hurisitic_and_cost_spent(child_cost)
                        child.set_depth(the_node.get_depth() + 1)
                        child.set_father(the_node)
                        heappush(frontier, child)           
    return None, len(explored)

A* with box-portal-goal heuristic

In [99]:
def solver_astar3(game_map, heuristic_func=heuristic2, weight=1):
    game = Game(game_map)
    root = State(game.get_player_position(),game.get_box_locations(),"")
    frontier = []
    explored = set()
    best_cost = {}
    root.set_depth(0)
    root_cost = weight * heuristic_func(game)
    root.set_sum_hurisitic_and_cost_spent(root_cost)
    root.set_father(None)
    heappush(frontier, root)
    best_cost[get_state_id(root.get_player_pos(), root.get_boxes_pos())] = root_cost
    while frontier:
        the_node = heappop(frontier)
        explored.add(get_state_id(the_node.get_player_pos(), the_node.get_boxes_pos()))
        game.set_player_position(the_node.get_player_pos())
        game.set_box_positions(the_node.get_boxes_pos())
        if game.is_game_won():
            return the_node.get_solution(), len(explored)
        for direction in ['U', 'D', 'R', 'L']:
            game.set_player_position(the_node.get_player_pos())
            game.set_box_positions(the_node.get_boxes_pos())
            result = game.apply_move(direction)
            if(result):
                child_state_id = get_state_id(game.get_player_position(), game.get_box_locations())
                if child_state_id not in explored:
                    child_cost = weight * heuristic_func(game) + the_node.get_depth() + 1
                    if child_state_id not in best_cost or child_cost < best_cost[child_state_id]:
                        best_cost[child_state_id] = child_cost
                        child = State(game.get_player_position(), game.get_box_locations(), direction)
                        child.set_sum_hurisitic_and_cost_spent(child_cost)
                        child.set_depth(the_node.get_depth() + 1)
                        child.set_father(the_node)
                        heappush(frontier, child)           
    return None, len(explored)

Weighted A*

In [100]:
def solver_astar4(game_map, heuristic_func=heuristic2, weight= 2.5):
    game = Game(game_map)
    root = State(game.get_player_position(),game.get_box_locations(),"")
    frontier = []
    explored = set()
    best_cost = {}
    root.set_depth(0)
    root_cost = weight * heuristic_func(game)
    root.set_sum_hurisitic_and_cost_spent(root_cost)
    root.set_father(None)
    heappush(frontier, root)
    best_cost[get_state_id(root.get_player_pos(), root.get_boxes_pos())] = root_cost
    while frontier:
        the_node = heappop(frontier)
        explored.add(get_state_id(the_node.get_player_pos(), the_node.get_boxes_pos()))
        game.set_player_position(the_node.get_player_pos())
        game.set_box_positions(the_node.get_boxes_pos())
        if game.is_game_won():
            return the_node.get_solution(), len(explored)
        for direction in ['U', 'D', 'R', 'L']:
            game.set_player_position(the_node.get_player_pos())
            game.set_box_positions(the_node.get_boxes_pos())
            result = game.apply_move(direction)
            if(result):
                child_state_id = get_state_id(game.get_player_position(), game.get_box_locations())
                if child_state_id not in explored:
                    child_cost = weight * heuristic_func(game) + the_node.get_depth() + 1
                    if child_state_id not in best_cost or child_cost < best_cost[child_state_id]:
                        best_cost[child_state_id] = child_cost
                        child = State(game.get_player_position(), game.get_box_locations(), direction)
                        child.set_sum_hurisitic_and_cost_spent(child_cost)
                        child.set_depth(the_node.get_depth() + 1)
                        child.set_father(the_node)
                        heappush(frontier, child)           
    return None, len(explored)

## Solve

In [101]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar,
    "A* without portal": solver_astar2,
    "A* optimal with portal": solver_astar3,
    "Weighted A*": solver_astar4
}

In [102]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [103]:
def solve_all_BFS():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        if map != 9:
            solve(map, "BFS")

solve_all_BFS()

BFS took 0.0 seconds on map 1 and visited 43 states.
6 moves were used: DURLLR
BFS took 0.0 seconds on map 2 and visited 21 states.
6 moves were used: LUDDUR
BFS took 0.0 seconds on map 3 and visited 121 states.
13 moves were used: ULDDUUUURDDDU
BFS took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
BFS took 0.1 seconds on map 5 and visited 6249 states.
15 moves were used: ULDDRDLLLUUURUR
BFS took 0.4 seconds on map 6 and visited 17530 states.
34 moves were used: UUUUURRRLLLLLLLDDDDDDDDDRDLURRRRRR
BFS took 20.11 seconds on map 7 and visited 632831 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLUUDD
BFS took 0.08 seconds on map 8 and visited 7492 states.
14 moves were used: UURDLDRRDRURDD
BFS took 5.37 seconds on map 10 and visited 241397 states.
46 moves were used: RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLR


In [104]:
def solve_all_DFS():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        if map < 5:
            solve(map, "DFS")

solve_all_DFS()

DFS took 0.0 seconds on map 1 and visited 11 states.
7 moves were used: UDDURLL
DFS took 0.0 seconds on map 2 and visited 9 states.
6 moves were used: LUDDUL
DFS took 0.0 seconds on map 3 and visited 139 states.
33 moves were used: ULUURDULDDDUUURDDUULDDDDUUUURDDDD
DFS took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!


In [105]:
def solve_all_IDS():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        if map < 6 or map == 8 :
            solve(map, "IDS")

solve_all_IDS()

IDS took 0.0 seconds on map 1 and visited 11 states.
7 moves were used: UDDURLL
IDS took 0.0 seconds on map 2 and visited 9 states.
6 moves were used: LUDDUL
IDS took 0.02 seconds on map 3 and visited 284 states.
13 moves were used: ULDDUUUURDDDD
IDS took 0.02 seconds on map 4 and visited 2 states.
No Solution Found!
IDS took 12.84 seconds on map 5 and visited 81510 states.
15 moves were used: ULDDRDLLLUUURUL
IDS took 13.87 seconds on map 8 and visited 46876 states.
14 moves were used: UURDLDRRDRURDR


In [106]:
def solve_all_astar():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        if map != 9:
            solve(map, "A*")

solve_all_astar()

A* took 0.0 seconds on map 1 and visited 44 states.
7 moves were used: UDLRDUR
A* took 0.0 seconds on map 2 and visited 21 states.
6 moves were used: LLRUDD
A* took 0.0 seconds on map 3 and visited 114 states.
13 moves were used: ULDDUUUURDDDD
A* took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
A* took 0.26 seconds on map 5 and visited 7250 states.
15 moves were used: ULDDRDLLLUUURUL
A* took 0.88 seconds on map 6 and visited 19850 states.
34 moves were used: DDDDDRRRLLLLLLLUUUUUUUURUULRRRRRRR
A* took 29.95 seconds on map 7 and visited 673760 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLUUDR
A* took 0.19 seconds on map 8 and visited 7855 states.
14 moves were used: UURDLDRRDRURDR
A* took 6.7 seconds on map 10 and visited 241697 states.
46 moves were used: RRRRRDRULURULLLDLLURUULDRDLDRRDRULURURDDRDLLLL


In [107]:
def solve_all_astar_optimal():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        if map != 9:
            solve(map, "A* without portal")

solve_all_astar_optimal()

A* without portal took 0.0 seconds on map 1 and visited 43 states.
7 moves were used: UDLRDUR
A* without portal took 0.0 seconds on map 2 and visited 19 states.
6 moves were used: LUDDUL
A* without portal took 0.0 seconds on map 3 and visited 75 states.
14 moves were used: ULUURDDDDUULDD
A* without portal took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
A* without portal took 0.03 seconds on map 5 and visited 668 states.
15 moves were used: LULDLRDRDLLULUU
A* without portal took 0.32 seconds on map 6 and visited 6595 states.
34 moves were used: UUUURURRLLLLLLLDDDRDDDDDDDLRRRRRRR
A* without portal took 2.1 seconds on map 7 and visited 40161 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU
A* without portal took 0.0 seconds on map 8 and visited 147 states.
19 moves were used: URRRRRRRRRRRRRURDDR
A* without portal took 1.16 seconds on map 10 and visited 56045 states.
46 moves were used: RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL


In [108]:
def solve_all_astar_more_optimal():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        if map != 9:
            solve(map, "A* optimal with portal")

solve_all_astar_more_optimal()

A* optimal with portal took 0.0 seconds on map 1 and visited 43 states.
7 moves were used: UDLRDUR
A* optimal with portal took 0.0 seconds on map 2 and visited 19 states.
6 moves were used: LUDDUL
A* optimal with portal took 0.0 seconds on map 3 and visited 75 states.
14 moves were used: ULUURDDDDUULDD
A* optimal with portal took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
A* optimal with portal took 0.01 seconds on map 5 and visited 668 states.
15 moves were used: LULDLRDRDLLULUU
A* optimal with portal took 0.18 seconds on map 6 and visited 6595 states.
34 moves were used: UUUURURRLLLLLLLDDDRDDDDDDDLRRRRRRR
A* optimal with portal took 2.23 seconds on map 7 and visited 40161 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU
A* optimal with portal took 0.01 seconds on map 8 and visited 231 states.
14 moves were used: UURDLDRRDRURDR
A* optimal with portal took 2.89 seconds on map 10 and visited 65295 states.
46 moves were used: RRRRRDRULURULLLULDRUUULDRDLDR

In [109]:
def solve_all_weighted_astar():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
            solve(map, "Weighted A*")
solve_all_weighted_astar()

Weighted A* took 0.0 seconds on map 1 and visited 14 states.
7 moves were used: UDDURLL
Weighted A* took 0.0 seconds on map 2 and visited 10 states.
6 moves were used: LUDDUL
Weighted A* took 0.0 seconds on map 3 and visited 26 states.
13 moves were used: ULDDUUUURDDDD
Weighted A* took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
Weighted A* took 0.0 seconds on map 5 and visited 70 states.
15 moves were used: LULDLRDRDLLULUU
Weighted A* took 0.07 seconds on map 6 and visited 2388 states.
34 moves were used: DDDDDRRRLLLLLLLUUUUUUURUUULRRRRRRR
Weighted A* took 0.19 seconds on map 7 and visited 4745 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLUUDR
Weighted A* took 0.01 seconds on map 8 and visited 164 states.
21 moves were used: URURDDDDLDRRRRLUURRDR
Weighted A* took 14.98 seconds on map 9 and visited 232578 states.
51 moves were used: RURRUUUULLLURRURDDDDDDRRDUURDRDLLRDLDLULLURLLURRURD
Weighted A* took 0.51 seconds on map 10 and visited 8349 states.
46 move

## A SUMMARY OF CONCLUSIONS

### Q0
**PROBLEM:** 
Why manhattan distance of player-box-goal is not good?

    1. where ever we have walls manhatten falls short so in maps with a lot of walls usually its not a very good method

    2. essential to note the existance of portals and consider them in our manhatten distance

**SOLUTION:**

one heuristic that works very good is for each box caluclate the function below and sum over all of them

min (manhat_dist_box_goal , manhat_dist_box_portin + manhat_dist_portout_goal , manhat_dist_box_portout + manhat_dist_portin_goal) 
        
this is an example for one port, two port is similar and the code is available.

(heuristic = 0 and heuristic = dist-box-goal have also been done.)

---

### Q1
**PROBLEM:**  
As you know, saving the whole map is very space and time consuming. For example, if we have multiple layers of walls, we don't care about the second layer of walls because we can't get past the first one! Not to mention, with each move, only a slight position change happens in the map — so the explored set and frontier sets would be very large in memory, and finding a specific element like the player's position would be very time consuming (searching each element in the map to find it).

**SOLUTION:**  
- Location of player  
- Location of boxes  
---

### Q2
**ACTIONS:**
- **UP**  
- **DOWN**  
- **LEFT**  
- **RIGHT**  

---

### Q3
**INITIAL STATE:**  
- Initial location of player and boxes  

**ACTIONS:**  
- **UP**, **DOWN**, **LEFT**, **RIGHT** 

**GOAL STATE:**
    All box locations match their respective goal locations
    
**PATH COST:**
    Each action costs 1 unit
    Total path cost = Total distance traveled

**TRANSITION MODEL:**  
If an action is possible (not colliding with walls or two (or more) boxes), then:  
- Update the player’s location  
- Check if there is a box in the player’s new location:  
  - If there is, update the box's location (it moves like the player if possible, meaning no walls or boxes block its path)  

```python
if action == possible:
    # Moving left
    left -> pos_player(x - 1, y)
    if there is box(pos_player_x, pos_player_y):
        update_box_location(x - 1, y)
    
    # Moving right
    right -> pos_player(x + 1, y)
    if there is box(pos_player_x, pos_player_y):
        update_box_location(x + 1, y)

    # Moving down
    down -> pos_player(x, y - 1)
    if there is box(pos_player_x, pos_player_y):
        update_box_location(x, y - 1)

    # Moving up
    up -> pos_player(x, y + 1)
    if there is box(pos_player_x, pos_player_y):
        update_box_location(x, y + 1)



### Q4  
**PROBLEM:**  

If we expand every state, then our search tree will have higher depth and branching factor, requiring more time and space.  
Another key factor is that we may get stuck in an infinite loop.  

**SOLUTION:**  

- **Forward-checking:** One way of reducing paths that will not lead to a solution is removing states in which 3 or 4 sides of a box are surrounded by walls or other boxes. Since the player cannot move these boxes, these states will never lead to a solution. By eliminating them, we not only remove these states but also all their potential child states.  
- **Avoiding infinite loops:** If we don't keep track of visited states and expand every neighbor, we might get stuck in an infinite loop. To prevent this, we mark visited paths and avoid re-adding a state unless it has a lower cost the second time; in that case, we update the state instead of duplicating it.  

---

### Q5  
**BFS**  

Pick a random node as a root. Expand this node and add all its neighbors to the frontier—this completes level 0.  
Pop from the frontier (FIFO list) and expand this node. After expanding every neighbor of the root, we have completed level 1 and so on.  

**Pros:**  
- Complete  
- Optimal  
- Good time complexity  
- Determines the minimum distance of every node from the root  

**Cons:**  
- Bad space complexity (must store all nodes at the last level, which grows exponentially)  

**DFS**  

Pick a random node as a root. Expand this node, then go to one of its neighbors and repeat—this searches by selecting a branch and going depth-first.  
despite it's cons, dfs is still used because it has good space complexity and for tree search because of not worring for loops.
**Pros:**  
- Good space complexity

**Cons:**
- Not Complete(if the number of states in total are finite and you handle loops then it would be complete)
- Not Optimal
- Bad time complexity (by limiting the depth in which it can go we can fix this (ids main idea))

**IDS**

removes the problem of space of bfs and time of dfs by using the best features of both of these, basically ids is limiting dfs in every state, meaning dfs but with max depth i and i in range(0, 1000) 1000 is an example and it matters for the user that how much cost it wants to pay.

**Pros:**  
- Complete  
- Optimal  
- Good time complexity  -> if b is branching factor and d the depth of optimal solution then (d+1)b + db^2 + (d-1)b^3 + ... + b^d = O(b^d)
- Good space complexity  -> O(bd) because we expand every node

**Astar**

by having an estimation of how close of we are to the goal(heuristic or h(n)) and the cost we have paid to get here(g(n)), A* is calculated(combining greedy best-fit search and bfs).

f(n) = h(n) + g(n)

expands the node with the lowest f(n) in the frontier

if heuristic is admissible and tree-search is done or consistant and graph-search -> optimal

---

### Q6
BFS provides optimal solution but number of visited states is much bigger.
DFS does not give optimal solutions and does not work really well in huge maps because of it's poor time complexity
IDS provides optimal solution and a bit better than dfs in some bigger maps but has time problem for others

---

### Q7
the heuristic I used is explained in the begining of the questions(title- middle part of project questions) and it is admissible because we are only moving in 
horizontal and vertical and never going back and because agent can only move in this 2 way then we have always predicted a smaller cost than the reality
(i am speaking about the final version, not considering the portals wouldn't result in an optimal solution because it's not admissible because the agent can use portals to shorten it's path and shorter than the manhatten distance).
it's not consistant because for example for 2 points the cost by heuristics can be greater than the real cost because the agent might use portals to shorten it.
for consistancy cost(a to c) >= h(a) - h(c)
